## Лабораторна робота №2: "Імплементація алгоритмів стиснення"

Склад команди та розподіл виконаних завдань:

-
-

Для кожного з алгоритмів поданих нижче
- опишіть як працює алгорит
- напишіть класи з методами encode та decode
- перевірте правильність кодування та декодування
- дослідіть час виконання коду в залежності від розмірів вхідних даних
- оцініть ступінь стиснення(у відсотка) в залежності від розмірів
- напишіть висновок про ефективність різних алгоритмів та умови за яких той чи інший алгоритм дають кращий результат

# Алгоритм Гаффмана

В цьому алгоритмі доцільно імплементувати клас node та додаткові функції в Huffman для побудови дерева кодування

In [43]:
import heapq 

class Node: 
    def __init__(self, freq, symbol, left=None, right=None): 
        self.frequency = freq

        self.symbol = symbol 
  
        self.left = left 
  
        self.right = right 
  
        self.huff = '' 
  
    def __lt__(self, nxt): 
        return self.frequency < nxt.frequency

class Huffman:
    def encode(self, text: str) -> tuple[str, dict[str, str]]:
        # sorting all characters in the text by frequency
        characters = {x: text.count(x) for x in sorted(list(text), key = lambda a:text.count(a))}
        nodes = []

        for ch, fr in characters.items():
            heapq.heappush(nodes, Node(fr, ch))

        while len(nodes) > 1:
            left = heapq.heappop(nodes)
            right = heapq.heappop(nodes)

            left.huff = 0
            right.huff = 1

            new_node = Node(left.frequency+right.frequency, left.symbol+right.symbol, left, right)

            heapq.heappush(nodes, new_node)

        result = {}
        def get_result(node, val=''):
            new_value = val + str(node.huff)
            if node.left:
                get_result(node.left, new_value)
            if node.right:
                get_result(node.right, new_value)
            if not node.left and not node.right:
                result[node.symbol] = new_value
        get_result(nodes[0])
        print(result)
        return ''.join(result[sym] for sym in text)

    def decode(self, code: str, coding_dict: dict[str, str]):
        message = ''
        current_code = ''
        for symbol in code:
            current_code += symbol
            for s, c in coding_dict.items():
                if c == current_code:
                    message += s
                    current_code = ''
        return message


In [44]:
import time
obj = Huffman()
count = 0

with open('example42.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()
    count = sum(1 for elem in file_content if elem in 'abc')

start = time.time()
encoded_result = obj.encode(file_content)
end = time.time()

final_time = end - start
print(final_time)
print(count)

{'b': '0', '\n': '100', 'a': '101', 'c': '11'}
932.2317171096802
572418


In [40]:
obj = Huffman()
count =0
with open('example42.txt', 'r', encoding='utf-8') as file:
    file = file.read()
    for elem in file:
        if elem in 'abc':
            count += 1
final = len(obj.encode())
res = final / len(obj) * 100
print(res)

100.0


# Алгоритм LZW

In [24]:
class LZW:
    def encode(self, text: str) -> tuple[str, list]:
        result = ''
        code_dict = {x[0]: x[1]+1 for x in zip(sorted(set(text)), range(len(set(text))))}
        next_index = len(code_dict) + 1
        current_string = ''
        for symbol in text:
            next_string = current_string + symbol
            if next_string in code_dict:
                current_string = next_string
            else:
                result += str(code_dict[current_string])
                code_dict[next_string] = next_index
                next_index += 1
                current_string = symbol
        if current_string in code_dict:
            result += str(code_dict[current_string])
        return result

    def decode(self, code: str, coding_dict: list) -> str:
        message = ''
        current_code = ''
        for symbol in code:
            current_code += symbol
            for s, c in coding_dict.items():
                if str(c) == current_code:
                    message += s
                    current_code = ''
        return message


In [29]:
import time

obj = LZW()
count = 0

with open('example42.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()
    for elem in file_content:
        if elem in 'abc':
            count += 1

start = time.time()
encoded_result = obj.encode(file_content)
end = time.time()

final_time = end - start
print(final_time)
print(count)

0.21686816215515137
572418


In [41]:
obj = LZW()
count =0
with open('example42.txt', 'r', encoding='utf-8') as file:
    file = file.read()
    for elem in file:
        if elem in 'abc':
            count += 1
final = len(obj.encode(file))
res = final / count * 100
print(res)

9.36780464625501


# Алгоритм LZ77

Потрібно заміряти розміри саме тексту, проте для роботи доцільно використовувати список тюплів, тому для зручності варто імплементувати додаткові алгоритми _text2list та _list2text

In [12]:
class LZ77:
    def __init__(self, buffer_size: int):
        self.buffer_size = buffer_size

    def encode(self, text):
        compressed_data = ''
        i = 0
        while i < len(text):
            match_length = 0
            match_offset = 0
            for j in range(1, min(self.buffer_size, len(text) - i) + 1):
                substring = text[i:i + j]
                previous_occurrence_index = max(i - self.buffer_size, 0)
                previous_occurrence = text[previous_occurrence_index:i]
                offset = previous_occurrence.rfind(substring)
                if offset != -1 and j > match_length:
                    match_length = j
                    match_offset = i - previous_occurrence_index - offset

            if match_length > 0:
                next_char_index = i + match_length
                next_char = text[next_char_index] if next_char_index < len(text) else ''
                compressed_data += f"({match_offset}, {match_length}, {next_char})"
                i += match_length + 1
            else:
                compressed_data += f"(0, 0, {text[i]})"
                i += 1

        return compressed_data
    def decode(self, code):
        decoded_text = ''
        i = 0
        while i < len(code):
            if code[i] == '(':
                j = i + 1
                while code[j] != ')':
                    j += 1
                triplet = code[i + 1:j]
                parts = triplet.split(', ')
                offset, length, next_char = int(parts[0]), int(parts[1]), parts[2]

                if offset == 0:
                    decoded_text += next_char
                else:
                    current_size = len(decoded_text)
                    for k in range(0, length):
                        decoded_text += decoded_text[current_size - offset + k]
                    decoded_text += next_char

                i = j + 1
            else:
                break

        return decoded_text
obj = LZ77(5)
text = 'abcacbabcabcabcacbacbacbbacaacbacbabcbacbabcbacba'
encoded_data = obj.encode(text)
print(encoded_data)
decoded_data = obj.decode(encoded_data)
print( decoded_data)
print(text)


(0, 0, a)(0, 0, b)(0, 0, c)(3, 1, c)(4, 1, a)(2, 1, c)(3, 3, a)(3, 3, c)(4, 1, a)(3, 3, c)(3, 1, b)(4, 2, a)(3, 2, b)(3, 3, a)(2, 1, c)(4, 2, c)(3, 2, b)(4, 3, c)(3, 2, )
abcacbabcabcabcacbacbacbbacaacbacbabcbacbabcbacba
abcacbabcabcabcacbacbacbbacaacbacbabcbacbabcbacba


In [13]:
import time
obj = LZ77(5)
count =0
with open('example42.txt', 'r', encoding='utf-8') as file:
    file = file.read()
    for elem in file:
        if elem in 'abc':
            count += 1
start = time.time()
obj.encode(file)
end = time.time()

final_time = end - start
print(final_time)
print(count)

3.4268877506256104
572418


In [17]:
obj = LZ77(5)
count =0
with open('example42.txt', 'r', encoding='utf-8') as file:
    file = file.read()
    for elem in file:
        if elem in 'abc':
            count += 1
final = len(obj.encode(file))
res = final / count * 100
print(res)

300.6774769486634


# Алгоритм Deflate

In [35]:
class Deflate:
    def __init__(self, buffer_size: int):
        self.buffer_size = buffer_size

    def encode(self, text: str) -> str:
        lz77 = LZ77(self.buffer_size).encode(text)
        return Huffman().encode(lz77.replace('(', '').replace(')','').replace(', ',''))
        

    def decode(self, code: str) -> str:
        pass

In [46]:
import time
obj = Deflate(5)

count = 0
with open('example42.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()
    for elem in file_content:
        if elem in 'abc':
            count += 1

start = time.time()
encoded_result = obj.encode(file_content)
end = time.time()

final_time = end - start

print(final_time)
print(count)

{'b': '00', '0': '010000', '\n': '010001', '5': '01001', 'c': '0101', '1': '011', '3': '10', '4': '110', '2': '1110', 'a': '1111'}
771.3616693019867
572418


In [38]:
obj = Deflate(5)
count =0
with open('example42.txt', 'r', encoding='utf-8') as file:
    file = file.read()
    for elem in file:
        if elem in 'abc':
            count += 1
final = len(obj.encode())
res = final / len() * 100
print(res)

{'3': '00', 'a': '010', '1': '011', '4': '100', 'b': '101', '0': '110', 'c': '1110', '2': '1111'}
381.25
